In [1]:
import os                      # For environment variables
from dotenv import load_dotenv # For loading .env file (API keys)
from langchain_community.vectorstores import Chroma         # Chroma vector DB
from langchain_openai import OpenAIEmbeddings, ChatOpenAI   # For LLM/embeddings
from langchain_core.prompts import ChatPromptTemplate       # Prompt templating

# Load secrets (should include OPENAI_API_KEY, etc.)
load_dotenv() 

True

In [2]:
CHROMA_PATH = "chroma"    # Path to your local Chroma DB

# 1. Create an embedding model (assumes OPENAI_API_KEY in .env)
embedding_function = OpenAIEmbeddings(
    api_key=os.environ.get("OPENAI_API_KEY"),
    model="text-embedding-3-small"
)

# 2. Load the persistent Chroma vector DB (for semantic search)
db = Chroma(
    persist_directory=CHROMA_PATH,
    embedding_function=embedding_function
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [3]:
from typing import Dict

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

def rag_query(question: str, similarity_threshold: float = 0.5) -> Dict:
    """
    Core RAG pipeline.
    - Retrieves relevant docs from vector DB using embeddings.
    - Builds context-aware prompt.
    - Calls chat LLM to generate answer.
    - Returns answer and document sources.
    """

    # 1. Semantic search in your vector DB
    results = db.similarity_search_with_relevance_scores(question, k=3)
    if not results or results[0][1] < similarity_threshold:
        return {"response": None, "sources": [], "found": False}

    # 2. Build the context for the prompt
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _ in results])

    # 3. Format the full prompt
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=question)

    # 4. Instantiate the LLM and ask the question
    llm = ChatOpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),
        model="gpt-4o-mini",  # Or any model you have access to
    )
    response_text = llm.predict(prompt)
    sources = [doc.metadata.get("source", None) for doc, _ in results]

    return {
        "response": response_text,
        "sources": sources,
        "context": context_text,
        "found": True
    }

In [4]:
user_question = "Does Alice falls through the earth"

result = rag_query(user_question)
if result["found"]:
    print("Answer:\n", result["response"])
    print("\nSources:\n", result["sources"])
else:
    print("No relevant context found.")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
C:\Users\Evan\AppData\Local\Temp\ipykernel_2108\1780745097.py:39: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response_text = llm.predict(prompt)


Answer:
 No, Alice does not fall through the earth. She wonders how far she has fallen and speculates about being near the center of the earth, but the context does not indicate that she actually falls through it.

Sources:
 ['data\\books\\alice_in_wonderland.md', 'data\\books\\alice_in_wonderland.md', 'data\\books\\alice_in_wonderland.md']
